In [18]:
# 0. 라이브러리 & 기본 설정
# =========================================
import torch
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import CrossEncoder, InputExample, losses
from torch.optim import AdamW
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, util
import warnings
from transformers import logging
import random

# 시드 고정
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

logging.set_verbosity_error()
warnings.filterwarnings("ignore")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [19]:
# =========================================
# 1. 학습 데이터셋 불러오기 (query, doc)
# =========================================
dataset = load_dataset("HJUNN/Art_Therapy_caption_train_dataset")
train_dataset = dataset["train"]      # features: ["query", "doc"]

df = train_dataset.to_pandas()
print("train 데이터 예시:")
print(df.head())

# 포지티브 쌍 리스트
positive_pairs = list(zip(df["query"], df["doc"]))
print(f"포지티브 샘플 수: {len(positive_pairs)}")

train 데이터 예시:
                                      query  \
0            집의 외관은 가족의 일상과 생활의 틀을 담고 있습니다.   
1                 지붕은 상상력과 생각의 세계를 펼쳐 보입니다.   
2                가족의 생활 패턴이 집의 구조에 녹아 있습니다.   
3  지붕이 화면을 가득 채우며, 그 크기가 비현실적으로 크게 표현되어 있다.   
4          그림의 대부분을 차지하는 거대한 지붕이 눈길을 사로잡는다.   

                                                 doc  
0  1. 제목: HTP : 집 집은 일상생활에서의 가정생활, 또는 가족 내에서의 자신에...  
1  1. 제목: HTP : 집 집은 일상생활에서의 가정생활, 또는 가족 내에서의 자신에...  
2  1. 제목: HTP : 집 집은 일상생활에서의 가정생활, 또는 가족 내에서의 자신에...  
3  ■ 과도하게 큰 지붕을 그린다 자신만의 환상을 가지고 있다 사회생활을 피하며 자신만...  
4  ■ 과도하게 큰 지붕을 그린다 자신만의 환상을 가지고 있다 사회생활을 피하며 자신만...  
포지티브 샘플 수: 573


In [20]:
# =========================================
# 2. 임베딩 모델로 하드 네거티브 생성
# =========================================
# (기존 bge-m3b 파인튜닝 임베딩 모델 사용)
embed_model = SentenceTransformer("HJUNN/bge-m3b-Art-Therapy-embedding-fine-tuning")

# 포지티브에 등장하는 doc 전체를 unique하게 정리
all_positive_docs = list(dict.fromkeys([d for _, d in positive_pairs]))
print("유니크 문서 수:", len(all_positive_docs))

# 전체 문서 임베딩
corpus_embeddings = embed_model.encode(
    all_positive_docs,
    convert_to_tensor=True,
    normalize_embeddings=True
)

# 쿼리별 네거티브 문서 매핑
from collections import defaultdict
neg_by_query = defaultdict(list)

print("하드 네거티브 생성 중...")
for q, pos_doc in tqdm(positive_pairs):
    q_emb = embed_model.encode(q, convert_to_tensor=True, normalize_embeddings=True)
    cos_scores = util.cos_sim(q_emb, corpus_embeddings)[0]

    # 상위 K개 후보 중에서 포지티브 doc을 제외한 것 중 일부를 하드 네거티브로 사용
    K = 10
    top_results = torch.topk(cos_scores, k=K)

    used = set()
    for idx in top_results.indices:
        candidate_doc = all_positive_docs[idx]
        if candidate_doc == pos_doc:
            continue
        if candidate_doc in used:
            continue
        neg_by_query[q].append(candidate_doc)
        used.add(candidate_doc)

print("쿼리별 하드 네거티브 예시:")
some_q = positive_pairs[0][0]
print("예시 쿼리:", some_q)
print("하드 네거티브 예시:", neg_by_query[some_q][:3])

유니크 문서 수: 191
하드 네거티브 생성 중...


  0%|          | 0/573 [00:00<?, ?it/s]

쿼리별 하드 네거티브 예시:
예시 쿼리: 집의 외관은 가족의 일상과 생활의 틀을 담고 있습니다.
하드 네거티브 예시: ['■ 집을 사람처럼 살아있다고 표현한다. 아동의 경우, 정상적인 경우가 있다. 청소년 및 성인의 경우, 어리광이 심하거나 또는 사회적으로 부적응하는 모습을 나타내고자 하는 경우가 있다. 선천적으로 신체적 어려움 및 정신적인 어려움을 겪고 있을 수 있다. 단 이것이 신체적 어려움 및 정신적 어려움을 판단하는 진단기준이 되어서는 안 된다.', '7. 집의 특징 이 외의 집을 표현하며 독특하고 특징적인 표현들을 관찰해야 한다. 집 틀의 빗물받이, 또는 홈통과 같은 파이프를 강조한다. 타인을 방어하고자 하거나 회피하려는 의심을 품고 있다. 집의 전체적인 모양이 부적절하게 틀어져 있고, 선이 구불거리며 각 집의 다양한 요소들이 어색하게 그려져 있다. 선천적으로 신체적 어려움 및 정신적인 어려움을 겪고 있을 수 있다. 단 이것이 신체적 어려움 및 정신적 어려움을 판단하는 진단기준이 되어서는 안 된다.', '■ 집의 청사진을 그리거나 집의 평면도를 그린다 집안 가족들과 지내면서 심각한 가족적인 문제를 겪고 있거나, 극도로 충격적인 사건을 겪은 적이 존재한다. 만약 청사진이나 평면도를 깔끔하고 완벽히 그린 경우, 극도로 무언가를 경계하며 의심하는 태도를 가지고 있다. (이와 같은 경우, "자신의 삶을 통제"하고 싶어하거나 "타인이 나에게 입히는 피해"에 집착할 수 있다.) 만약 청사진이나 평면도를 흐리멍덩하거나 부적절하게 그린 경우, 선천적으로 신체적 어려움 및 정신적인 어려움을 겪고 있을 수 있다. 단 이것이 신체적 어려움 및 정신적 어려움을 판단하는 진단기준이 되어서는 안 된다.']


In [21]:
# =========================================
# 3. (query, positive_doc, negative_doc) 트리플 생성
# =========================================
triples = []

for q, pos_doc in positive_pairs:
    neg_list = neg_by_query[q]
    if not neg_list:
        # 해당 쿼리에 하드 네거티브가 없으면 스킵
        continue

    # 1~3개 정도 네거티브를 뽑아서 여러 트리플로 만들 수도 있음
    num_negs = min(3, len(neg_list))
    sampled_negs = random.sample(neg_list, num_negs)

    for neg_doc in sampled_negs:
        triples.append((q, pos_doc, neg_doc))

print("생성된 (query, pos, neg) 트리플 수:", len(triples))
print("트리플 예시 1개:")
print(triples[0])


생성된 (query, pos, neg) 트리플 수: 1719
트리플 예시 1개:
('집의 외관은 가족의 일상과 생활의 틀을 담고 있습니다.', '1. 제목: HTP : 집 집은 일상생활에서의 가정생활, 또는 가족 내에서의 자신에 대한 인식을 나타낸다. 자신의 현실의 모습일 수도 있고, 또는 자신이 바라는 모습, 또는 가족의 생활패턴을 나타낸다. 지붕 지붕의 핵심은 머리로 상상할 수 있는 생각을 나타낸다.', '7. 집의 특징 이 외의 집을 표현하며 독특하고 특징적인 표현들을 관찰해야 한다. 집 틀의 빗물받이, 또는 홈통과 같은 파이프를 강조한다. 타인을 방어하고자 하거나 회피하려는 의심을 품고 있다. 집의 전체적인 모양이 부적절하게 틀어져 있고, 선이 구불거리며 각 집의 다양한 요소들이 어색하게 그려져 있다. 선천적으로 신체적 어려움 및 정신적인 어려움을 겪고 있을 수 있다. 단 이것이 신체적 어려움 및 정신적 어려움을 판단하는 진단기준이 되어서는 안 된다.')


In [22]:
# =========================================
# 4. Pairwise Ranking Dataset
# =========================================
class PairwiseRankDataset(Dataset):
    def __init__(self, triples, tokenizer, max_length=256):
        self.triples = triples
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.triples)

    def __getitem__(self, idx):
        query, pos_doc, neg_doc = self.triples[idx]

        pos = self.tokenizer(
            query, pos_doc,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        neg = self.tokenizer(
            query, neg_doc,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            "pos_input_ids": pos["input_ids"].squeeze(0),
            "pos_attention_mask": pos["attention_mask"].squeeze(0),
            "neg_input_ids": neg["input_ids"].squeeze(0),
            "neg_attention_mask": neg["attention_mask"].squeeze(0),
        }

In [23]:
# =========================================
# 5. BGE Reranker v2 m3 로드
# =========================================
model_name = "BAAI/bge-reranker-v2-m3"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=1   # 점수 1개 출력
)
model.to(device)

# dataset & dataloader
dataset = PairwiseRankDataset(triples, tokenizer, max_length=256)
batch_size = 4   # ⚠ bge-v2-m3는 heavy → batch 줄이기 추천
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

In [24]:
# =========================================
# 6. Optimizer & Loss
# =========================================
optimizer = AdamW(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss()


# =========================================
# 7. 학습 루프 (Pairwise Ranking)
# =========================================
epochs = 2   # ⚠ GPU 메모리 고려해 적은 epoch부터

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):

        pos_ids = batch["pos_input_ids"].to(device)
        pos_mask = batch["pos_attention_mask"].to(device)

        neg_ids = batch["neg_input_ids"].to(device)
        neg_mask = batch["neg_attention_mask"].to(device)

        optimizer.zero_grad()

        pos_out = model(input_ids=pos_ids, attention_mask=pos_mask)
        pos_score = pos_out.logits   # (B,1)

        neg_out = model(input_ids=neg_ids, attention_mask=neg_mask)
        neg_score = neg_out.logits   # (B,1)

        logits = torch.cat([pos_score, neg_score], dim=1)  # (B,2)
        labels = torch.zeros(logits.size(0), dtype=torch.long).to(device)

        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"[Epoch {epoch+1}] Loss: {total_loss/len(dataloader):.4f}")

print("학습 완료!")

Epoch 1/2:   0%|          | 0/430 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 6.12 MiB is free. Process 9885 has 14.73 GiB memory in use. Of the allocated memory 14.57 GiB is allocated by PyTorch, and 41.17 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# =========================================
# 8. 평가 함수
# =========================================
def calculate_similarity(model, tokenizer, query, candidates):
    model.eval()
    results = []

    with torch.no_grad():
        for doc in candidates:
            enc = tokenizer(
                query, doc,
                padding="max_length",
                truncation=True,
                max_length=256,
                return_tensors="pt"
            ).to(device)

            score = model(**enc).logits.squeeze().item()
            results.append((doc, score))

    return sorted(results, key=lambda x: x[1], reverse=True)


In [ ]:
# =========================================
# 9. 검증 데이터 평가
# =========================================
valid_dataset = load_dataset("HJUNN/Art_Therapy_caption_valid_dataset")["validation"]
valid_df = valid_dataset.to_pandas()

test_queries = [
    valid_df["query"][0],
    valid_df["query"][55],
    valid_df["query"][85]
]

test_candidates = [
    valid_df["doc"][0], valid_df["doc"][7], valid_df["doc"][20],
    valid_df["doc"][55], valid_df["doc"][60], valid_df["doc"][65],
    valid_df["doc"][85], valid_df["doc"][90], valid_df["doc"][95],
]

print("\n=== 학습 후 Reranker 점수 ===")
for q in test_queries:
    print(f"\n[Query] {q}")
    scores = calculate_similarity(model, tokenizer, q, test_candidates)
    for i, (doc, s) in enumerate(scores, start=1):
        print(f"{i}위 | 점수 {s:.4f} | 문서: {doc[:60]}...")
    print("-" * 70)

model.save_pretrained("./htp_bge_cross_encoder")
tokenizer.save_pretrained("./htp_bge_cross_encoder")
print("저장 완료: ./htp_bge_cross_encoder")
